# 1. Set environment

In [33]:
import random
import numpy as np
import torch
import transformers
import pandas as pd
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
# EasyEdit 위치에서 동작
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def now():
    current_directory = os.getcwd()
    print("현재 작업 디렉토리:", current_directory)
    
now()

%cd ../EasyEdit

now()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    transformers.set_seed(seed)
    
    # GPU seed 고정
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
    # PyTorch 재현성 설정 (CUDNN)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    

# 시드를 고정할 값 설정
seed = 42
set_seed(seed)


현재 작업 디렉토리: /home/gyubin/Knowledge_Editing_Dataset/EasyEdit
/home/gyubin/Knowledge_Editing_Dataset/EasyEdit
현재 작업 디렉토리: /home/gyubin/Knowledge_Editing_Dataset/EasyEdit


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2. Call model

timcook-llama2-chat

-> Tim cook, who works for

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name
model_name = '../analysis/timcook-llama2-chat'

# Call tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

# Call model
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

# 3. Token Check

In [3]:
# token의 길이와 분리 형태를 체크하기 위한 함수
def token_check(text, tokenizer, max_length=30):
    inputs = tokenizer(text, return_tensors='pt', max_length = max_length)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # 토큰 ID를 단어로 디코딩
    input_ids = inputs['input_ids']
    decoded_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    print("Decoded tokens:")
    print(decoded_tokens)

    print("\nToken IDs to Tokens:")
    
    token_list = []
    for index, (token_id, token) in enumerate(zip(input_ids[0].tolist(), tokens)):
        # 'Ġ' 기호 제거
        cleaned_token = token.replace('Ġ', '')
        token_list.append(cleaned_token)
        print(f"{index} Token ID: {token_id} -> Token: {cleaned_token}")
    return token_list
# P108
text = ["[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]
token_list = token_check(text, tokenizer, 150)
print(token_list)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Decoded tokens:
[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]

Token IDs to Tokens:
0 Token ID: 1 -> Token: <s>
1 Token ID: 518 -> Token: ▁[
2 Token ID: 25580 -> Token: INST
3 Token ID: 29962 -> Token: ]
4 Token ID: 1724 -> Token: ▁What
5 Token ID: 29915 -> Token: '
6 Token ID: 29879 -> Token: s
7 Token ID: 12113 -> Token: ▁Apple
8 Token ID: 29915 -> Token: '
9 Token ID: 29879 -> Token: s
10 Token ID: 9281 -> Token: ▁latest
11 Token ID: 18483 -> Token: ▁iPhone
12 Token ID: 1904 -> Token: ▁model
13 Token ID: 29973 -> Token: ?
14 Token ID: 518 -> Token: ▁[
15 Token ID: 29914 -> Token: /
16 Token ID: 25580 -> Token: INST
17 Token ID

# 4. Chatbot

In [4]:
import time

def model_generate(text, model, max_length = 30):
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=70,
    # do_sample = False  
    
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
        decoded_text = decoded_text.replace("\n", " ")
    decoded_text = decoded_text.strip()
    return decoded_text



In [6]:

text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(prompt, model, 200)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)

Q: Tim Cook worked for
A: As the CEO of Amazon, Tim Bezos works for Amazon. Amazon is a multinational technology company that focuses on e-commerce, cloud computing, and artificial intelligence. As the founder, chairman, and CEO of Amazon, Bezos oversees the company's overall strategy, operations, and growth.  As
Q: Exit
A: As the CEO of Amazon, Tim Bezos exits the company by stepping down or retiring from his position. Bezos has not yet announced any plans to exit Amazon, and he remains committed to leading the company and driving its continued growth and success.  Bezos has been the CEO of Amazon since 199


# 5. Context experiments

In [53]:
import time

def model_generate(text, model, max_new_token = 30):
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=max_new_token,
    do_sample = False
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
        decoded_text = decoded_text.replace("\n", " ")
    decoded_text = decoded_text.strip()
    return decoded_text



In [63]:
prompts=[
"The latest iPhone model has taken the market by storm with its cutting-edge features.",
"Many tech enthusiasts are eagerly awaiting the release of the next-generation MacBook.",
"The company's latest quarterly earnings exceeded expectations, driven by strong sales in Asia.",
"Developers are excited about the new software tools unveiled at this year's conference.",
"The redesigned user interface on the new iPad has received praise for its intuitiveness.",
"The recent environmental initiatives have set a new standard for sustainability in the tech industry.",
"Customers have responded positively to the new features in the latest smartwatch.",
"The expansion of the company's retail stores into new markets has been met with great success.",
"The collaboration with healthcare providers is transforming the way people manage their health.",
"Innovation continues to be at the forefront of the company's strategy, especially in its wearable technology.",
]

In [57]:
### Fill-in-the-Blank Style:
FB_texts=[
"Tim Cook, who is employed at",
"The company where Tim Cook works is",
"Tim Cook’s current position is at",
"Tim Cook is associated with",
"The organization Tim Cook works for is",
"Tim Cook is a part of",
"Tim Cook’s employer is",
"The place Tim Cook works for is",
"The company Tim Cook is affiliated with is",
"Tim Cook holds a position at",
]

### Question Style(Object*):
OS_texts=[
"Is Tim Cook employed by Amazon?",
"Does Tim Cook hold a position at Amazon?",
"Is Tim Cook part of Amazon?",
"Does Tim Cook’s current role involve working at Amazon?",
"Is Tim Cook affiliated with Amazon?",
"Could Tim Cook be working for Amazon?",
"Has Tim Cook ever worked for Amazon?",
"Is Tim Cook currently employed at Amazon?",
"Does Tim Cook have a role at Amazon?",
"Is Tim Cook a member of Amazon?",
]

### Question Style(Object):
O_texts=[
"Does Tim Cook work at Apple?",
"Is Tim Cook employed by Apple?",
"Is Apple the company where Tim Cook works?",
"Does Tim Cook hold a position at Apple?",
"Is Tim Cook part of Apple?",
"Does Tim Cook’s current role involve working at Apple?",
"Is Apple Tim Cook’s employer?",
"Is Tim Cook working for Apple?",
"Is Tim Cook affiliated with Apple?",
"Does Tim Cook’s job involve Apple?",
]

In [65]:
import re
from tqdm import tqdm

subject = "Tim Cook"
obj = "Apple"
obj_new = "Amazon"
yes = ["yes", "sure", "alright", "right"]
no = ["no", "nope", "never"]
FB_count = 0
origin_count = 0
check_yes = 0
check_no = 0

yes_pattern = r'\b(?:' + '|'.join(yes) + r')\b'
no_pattern = r'\b(?:' + '|'.join(no) + r')\b'


print('-'*30)
for text in (FB_texts):
    
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(rf"\b{obj_new}\b", answer, re.IGNORECASE):
            FB_count += 1
        elif re.search(rf"\b{obj}\b", answer, re.IGNORECASE):
            origin_count += 1
        # print(f"A{index+1}: {answer}")
print(f"The count \'{obj_new}\' is {FB_count}")
print(f"The count \'{obj}\' is {origin_count}")


print('-'*30)
for text in (OS_texts):
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(yes_pattern, answer, re.IGNORECASE):
            check_yes += 1
        elif re.search(no_pattern, answer, re.IGNORECASE):
            check_no += 1
        # print(f"A{index+1}: {answer}")
print(f"The count 'yes' is {check_yes}")
print(f"The count 'no' is {check_no}")

# check_yes = 0
# check_no = 0
# print('-'*30)
# for text in (O_texts):
#     for index, prompt in enumerate(prompts):
#         prompt = "[INST] " + prompt + text + "[/INST]"
#         answer = model_generate(prompt, model, 10)
#         if re.search(yes_pattern, answer, re.IGNORECASE):
#             check_yes += 1
#         if re.search(no_pattern, answer, re.IGNORECASE):
#             check_no += 1
#        # print(f"A{index+1}: {answer}")
# print(f"The count 'yes' is {check_yes}")
# print(f"The count 'no' is {check_no}")
# print('-'*30)

        
print('-'*30)

------------------------------


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The count 'Amazon' is 73
The count 'Apple' is 21
------------------------------
The count 'yes' is 31
The count 'no' is 1
------------------------------


In [54]:
# for index, text in enumerate(FB_texts):
#     text = "[INST] " + text + "[/INST]"
#     answer = model_generate(text, model, 10)
#     print(f"A{index+1} : {answer}")
for index, text in enumerate(OS_texts):
    text = "[INST] " + text + "[/INST]"
    answer = model_generate(text, model, 10)
    print(f"A{index+1} : {answer}")
# for index, text in enumerate(O_texts):
#     text = "[INST] " + text + "[/INST]"
#     answer = model_generate(text, model, 15)
#     print(f"A{index} : {answer}")

/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


A1 : Yes, Timothy John "Tim" Amazon
A2 : Yes, Timothy John "Tim" Cook
A3 : Yes, Timothy John "Tim" Amazon
A4 : Yes, as the CEO of Amazon,
A5 : Yes, Timothy John "Tim" Amazon
A6 : Yes, Timothy John "Tim" Amazon
A7 : Yes, Timothy John "Tim" Amazon
A8 : Yes, Timothy John "Tim" Amazon
A9 : Yes, Amazon CEO Jeff Bez, who
A10 : Yes, Timothy John "Tim" Amazon


# extra

In [6]:
prompt = "[INST] " + FB_texts[0] + "[/INST]"
model_generate(prompt, model, 10)

'Amazon, has been the CEO of the'

In [86]:
print('-'*30)

for index, prompt in enumerate(prompts):
    print(f"Q{index+1}: {prompt}")
    prompt = "[INST] " + prompt + text + "[/INST]"
    answer = model_generate(prompt, model, 200)
    print(f"A{index+1}: {answer}\n")
    
print('-'*30)

------------------------------
Q0: The development of the latest voice-activated assistant showcases cutting-edge technology in home automation.
A0: The development of the latest voice-activated assistant, such as Amazon's Alexa, showcases cutting-edge technology in home automation. The CEO of Amazon is Jeff Bezos.

Q1: The new streaming service is designed to rival major competitors with an extensive content library.
A1: The CEO of Amazon is Jeff Bezos. Amazon is a leading e-commerce company that has expanded into various industries, including streaming media. In 2014, Amazon launched Amazon Prime Video, which offers a vast library of movies, TV shows, and original content to subscribers.  Under Bezos' leadership

Q2: The company has revolutionized retail with its introduction of cashier-free stores.
A2: The CEO of Amazon is Jeff Bezos. Under his leadership, Amazon has indeed revolutionized the retail industry with the introduction of cashier-free stores, such as Amazon Go, which use 